In [1]:
from datetime import datetime, timedelta
from selenium import webdriver
import time
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
first_part_url = "https://www.kiwi.com/en/search/results/"
last_part_url = "/"
# https://www.kiwi.com/en/search/results/paris-france/london-united-kingdom/2025-03-02/2025-03-03/

In [3]:
TTT = [(datetime.today() + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, 31)]

In [4]:
LOS = [(datetime.today() + timedelta(days=i)).strftime('%Y-%m-%d') for i in range(1, 6)]

In [5]:
snapshot_date = datetime.today().strftime('%Y-%m-%d')

In [6]:
dep = ['london-united-kingdom' , 'paris-france' , 'rome-italy' ]
back = ['london-united-kingdom' , 'paris-france' , 'rome-italy' ]

In [7]:
all_urls = []

# Outer loops over the city combinations
for origin in dep:
    for destination in back:
        if origin != destination:
            # For a given city pair, iterate over all departure dates
            for t in TTT:
                # For each departure date, create return dates offset from +1 to +5 days
                for delta in range(1, 6):
                    return_date = (datetime.strptime(t, "%Y-%m-%d") + timedelta(days=delta)).strftime("%Y-%m-%d")
                    # Construct the URL including departure and return dates
                    url = f"{first_part_url}{origin}/{destination}/{t}/{return_date}{last_part_url}"
                    all_urls.append((url, t, return_date))

# Display the total count and a few sample URLs
print(f"Total URLs generated: {len(all_urls)}")
print("Sample URLs:")
for u in all_urls[:]:
    print(u)

Total URLs generated: 900
Sample URLs:
('https://www.kiwi.com/en/search/results/london-united-kingdom/paris-france/2025-03-03/2025-03-04/', '2025-03-03', '2025-03-04')
('https://www.kiwi.com/en/search/results/london-united-kingdom/paris-france/2025-03-03/2025-03-05/', '2025-03-03', '2025-03-05')
('https://www.kiwi.com/en/search/results/london-united-kingdom/paris-france/2025-03-03/2025-03-06/', '2025-03-03', '2025-03-06')
('https://www.kiwi.com/en/search/results/london-united-kingdom/paris-france/2025-03-03/2025-03-07/', '2025-03-03', '2025-03-07')
('https://www.kiwi.com/en/search/results/london-united-kingdom/paris-france/2025-03-03/2025-03-08/', '2025-03-03', '2025-03-08')
('https://www.kiwi.com/en/search/results/london-united-kingdom/paris-france/2025-03-04/2025-03-05/', '2025-03-04', '2025-03-05')
('https://www.kiwi.com/en/search/results/london-united-kingdom/paris-france/2025-03-04/2025-03-06/', '2025-03-04', '2025-03-06')
('https://www.kiwi.com/en/search/results/london-united-kin

In [8]:
driver = webdriver.Chrome()
driver.get(all_urls[0][0])

In [ ]:
while True:
    # Find result elements
    elements = driver.find_elements(By.XPATH, './/*[@class="group/result-card relative cursor-pointer leading-normal"]')

    # Stop if we have 100 or more results
    if len(elements) >= 100:
        break

    try:
        # Find the "Load More" button
        load_more_button = WebDriverWait(driver, 40).until(
            EC.element_to_be_clickable((By.XPATH, "//button[div[contains(text(), 'Load more')]]"))
        )
        load_more_button.click()
        time.sleep(1)  # Small delay to allow results to load
    except:
        break  # If the button is missing or unclickable, exit the loop

# Print the total number of elements found
print("Total results loaded:", len(elements))

In [19]:
for item in driver.find_elements(By.XPATH, './/*[@class="group/result-card relative cursor-pointer leading-normal"]'):
        duration = item.find_element(By.XPATH, './/*[@class="flex flex-grow flex-col justify-center"]').text
        price = item.find_element(By.XPATH, './/*[@class="lm:mb-300 lm:flex lm:flex-auto lm:flex-col lm:justify-center lm:text-center"]').text
        print (duration, price)

Outbound
07:55
1h 25m
10:20
LGW
Direct
ORY ₪ 670
Outbound
07:55
1h 25m
10:20
LGW
Direct
ORY ₪ 674
Outbound
07:55
1h 25m
10:20
LGW
Direct
ORY ₪ 670


In [57]:
with open('flight_data_kiwi.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Snap Date","Dep Date","Return Date","Outbound","Dep Time","Dep Duration","Dep Arrival","From","Stops","Landing","Inbound","Back Time","Back Duration","Back Arrival","From","Stops","Landing","Dep Company","Back Company","Price"])

    for url, dep_date, return_date in all_urls:
        driver.get(url)
        time.sleep(3)
        
        # Click 'Show More' button until 100 results are present
        while True:
        # Find result elements
            elements = driver.find_elements(By.XPATH, './/*[@class="group/result-card relative cursor-pointer leading-normal"]')

            # Stop if we have 100 or more results
            if len(elements) >= 100:
                break

            try:
            # Find the "Load More" button
                load_more_button = WebDriverWait(driver, 45).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[div[contains(text(), 'Load more')]]"))
                )
                load_more_button.click()
                time.sleep(1)  # Small delay to allow results to load
            except:
                break  # If the button is missing or unclickable, exit the loop

        
        # Extracting flight details
        for item in driver.find_elements(By.XPATH, './/*[@class="group/result-card relative cursor-pointer leading-normal"]'):
            duration_dep = item.find_element(By.XPATH, './/*[@class="relative z-default flex h-full flex-col"]').text
            duration_parts = duration_dep.split("\n")  # Splitting by newline to commas.
            #Removing irrelevant terms "Throwaway ticketing hack" | bags | cabins if it exists. הייתה פה חשיבה מחוץ לקופסא
            duration_parts = [part for part in duration_parts if part.strip() not in ("Throwaway ticketing hack","Verified by Ryanair","+1","1","0","Self-transfer hack","Hidden cities hack",1,+1,0)]

            image_element = driver.find_element(By.XPATH, '//*[@class="max-w-none bg-transparent rounded-100 last:self-end h-icon-large w-icon-large"]') 
            dep_back_companies = image_element.get_attribute("title")
            companies_parts = dep_back_companies.split("\n")
            price = item.find_element(By.XPATH, './/*[@class="lm:mb-300 lm:flex lm:flex-auto lm:flex-col lm:justify-center lm:text-center"]').text
            writer.writerow([snapshot_date,dep_date, return_date] + duration_parts + companies_parts + [price])

# Close WebDriver
driver.quit()

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=133.0.6943.142); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7A4E2C6A5+28789]
	(No symbol) [0x00007FF7A4D95B20]
	(No symbol) [0x00007FF7A4C28F9A]
	(No symbol) [0x00007FF7A4C3EDA4]
	(No symbol) [0x00007FF7A4C3D873]
	(No symbol) [0x00007FF7A4C316D1]
	(No symbol) [0x00007FF7A4C2F402]
	(No symbol) [0x00007FF7A4C3376C]
	(No symbol) [0x00007FF7A4C3383F]
	(No symbol) [0x00007FF7A4C78F86]
	(No symbol) [0x00007FF7A4CA732A]
	(No symbol) [0x00007FF7A4C71BC6]
	(No symbol) [0x00007FF7A4CA7540]
	(No symbol) [0x00007FF7A4CCF7E3]
	(No symbol) [0x00007FF7A4CA7103]
	(No symbol) [0x00007FF7A4C6FFC0]
	(No symbol) [0x00007FF7A4C71273]
	GetHandleVerifier [0x00007FF7A5171AED+3458237]
	GetHandleVerifier [0x00007FF7A518829C+3550316]
	GetHandleVerifier